In [1]:
import yfinance as yf

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)

# Data

In [2]:
def open_df(name, date_from, date_to):
    return yf.download(name, date_from, date_to, interval='1h').reset_index()

print('Downloading Brent data')
df_brent = open_df('BZ=F', "2021-01-01", "2022-12-12")
print('Downloading USD/RUB data')
df_usd = open_df('RUB=X', "2021-01-01", "2022-12-12")

df = pd.merge(df_usd, df_brent, on='index')
df.drop(columns=['Volume_x'], inplace=True)
df.set_index('index', inplace=True)
df.shape

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


(8867, 11)

In [3]:
train_size = 7000

In [4]:
def sliding_window_features(df, target_column, size):
    df_new = df.sort_index(ascending=True).copy()

    columns = df.columns
    feature_columns = list(set(columns) - {target_column,})
    for column in columns:
        for shift in range(1, size + 1):
            feature_values = df_new[column].values
            feature_values_shifted = np.roll(feature_values, -shift)
            df_new[column + '_' + str(shift)] = feature_values_shifted

    y = df_new[target_column]

    df_new = df_new.drop(columns, axis=1)

    res_df = df_new.iloc[:-size]
    res_y = pd.Series(np.roll(y.diff(), -1)[:-size], index=res_df.index)
    return res_df, res_y

# Baseline

In [5]:
from catboost import CatBoostRegressor

widow_size = 1
X, y = sliding_window_features(df, 'Close_x', widow_size)
cb = CatBoostRegressor()
cb.fit(X[:train_size], y[:train_size], verbose=False)
print(f'RMSE:', np.sqrt(np.mean((y[train_size:] - cb.predict(X[train_size:]))**2)))

RMSE: 0.588099889393787


In [6]:
from catboost import CatBoostRegressor

widow_size = 5
X, y = sliding_window_features(df, 'Close_x', widow_size)
cb = CatBoostRegressor()
cb.fit(X[:train_size], y[:train_size], verbose=False)
print(f'RMSE:', np.sqrt(np.mean((y[train_size:] - cb.predict(X[train_size:]))**2)))

RMSE: 0.5912915270204362


In [7]:
from catboost import CatBoostRegressor

widow_size = 10
X, y = sliding_window_features(df, 'Close_x', widow_size)
cb = CatBoostRegressor()
cb.fit(X[:train_size], y[:train_size], verbose=False)
print(f'RMSE:', np.sqrt(np.mean((y[train_size:] - cb.predict(X[train_size:]))**2)))

/var/folders/lw/v8xx90_s5mj452fmqx1ltbtr0000gn/T/ipykernel_16436/2899716042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column + '_' + str(shift)] = feature_values_shifted
/var/folders/lw/v8xx90_s5mj452fmqx1ltbtr0000gn/T/ipykernel_16436/2899716042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column + '_' + str(shift)] = feature_values_shifted
/var/folders/lw/v8xx90_s5mj452fmqx1ltbtr0000gn/T/ipykernel_16436/2899716042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

RMSE: 0.5742990231469814


# RMDN (Recurrent Mixture Density Network)

In [8]:
class FinancialDataset(Dataset):
    def __init__(self, X, y, sequence_length):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.y) - self.sequence_length

    def __getitem__(self, idx):
        return self.X[idx:idx+self.sequence_length], self.y[idx+self.sequence_length]

X, y = sliding_window_features(df, 'Close_x', 1)

X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

In [9]:
widow_size = 1

train_dataset1 = FinancialDataset(X_train, y_train, widow_size)
test_dataset1 = FinancialDataset(X_test, y_test, widow_size)

train_dataloader1 = DataLoader(train_dataset1, batch_size=16, shuffle=False)
test_dataloader1 = DataLoader(test_dataset1, batch_size=16, shuffle=False)

In [10]:
widow_size = 5

train_dataset5 = FinancialDataset(X_train, y_train, widow_size)
test_dataset5 = FinancialDataset(X_test, y_test, widow_size)

train_dataloader5 = DataLoader(train_dataset5, batch_size=16, shuffle=False)
test_dataloader5 = DataLoader(test_dataset5, batch_size=16, shuffle=False)

In [11]:
widow_size = 10

train_dataset10 = FinancialDataset(X_train, y_train, widow_size)
test_dataset10 = FinancialDataset(X_test, y_test, widow_size)

train_dataloader10 = DataLoader(train_dataset10, batch_size=16, shuffle=False)
test_dataloader10 = DataLoader(test_dataset10, batch_size=16, shuffle=False)

In [12]:
class MDN(nn.Module):
    def __init__(self, in_features, num_distributions):
        super(MDN, self).__init__()
        self.in_features = in_features
        self.num_distributions = num_distributions
        
        self.pi = nn.Linear(in_features, num_distributions)
        self.mu = nn.Linear(in_features, num_distributions)
        self.sigma = nn.Linear(in_features, num_distributions)
        self.elu = nn.ELU()

    def forward(self, x):
        pi = F.gumbel_softmax(self.pi(x), dim=-1, tau=1)
        mu = self.mu(x)
        sigma = self.elu(self.sigma(x)) + 1 + 1e-15
        return pi, mu, sigma


class RMDN(nn.Module):
    def __init__(self, in_features=11, hidden_size=32, num_distributions=2):
        super(RMDN, self).__init__()
        self.in_features = in_features
        self.hidden_size = hidden_size
        self.num_distributions = num_distributions
        
        self.GRU = nn.GRU(in_features, hidden_size, batch_first=True)
        self.MDN = MDN(hidden_size, num_distributions)
    
    def forward(self, x):
        h0 = torch.autograd.Variable(torch.zeros(1, x.shape[0], self.hidden_size))
        out, h = self.GRU(x, h0)
        out = out[:,-1,:]
        pi, mu, sigma = self.MDN(out)
        return pi, mu, sigma

In [13]:
def test_rmse(model, test_dataset, test_dataloader):
    mse = 0
    with torch.inference_mode():
        for batch_x, batch_y in tqdm(test_dataloader, desc='Test'):
            pi, mu, sigma = model(batch_x)
            value = torch.sum(pi * mu, dim=1)  # expected values
            err = batch_y - value
            mse += torch.sum(err**2)
    return torch.sqrt(mse / len(test_dataset))

In [14]:
ONEOVERSQRT2PI = 1.0 / np.sqrt(2 * np.pi)
LOG2PI = np.log(2 * np.pi)

def log_pdf(sigma, mu, target):
    target = target.expand_as(sigma)
    return -torch.log(sigma) - 0.5 * LOG2PI - 0.5 * torch.pow((target - mu) / sigma, 2)

def log_prob_y(pi, sigma, mu, y):
    log_component_prob = log_pdf(sigma, mu, y)
    log_mix_prob = torch.log(pi + 1e-15)
    return torch.logsumexp(log_component_prob + log_mix_prob, dim=-1)

def calculate_loss(y, pi, mu, sigma, model, lambda_pi=1.0):
    log_prob = log_prob_y(pi, sigma, mu, y)
    loss = torch.mean(-log_prob)
    
    pi_l1_reg = 0
    if lambda_pi > 0:
        pi_params = torch.cat([x.view(-1) for x in model.MDN.pi.parameters()])
        pi_l1_reg = lambda_pi * torch.norm(pi_params, 1)

    loss = loss + pi_l1_reg
    return loss

Обучение

In [15]:
num_distributions = 3
hidden_size = 4
lambda_pi = 1
EPOCHS = 10

model1 = RMDN(num_distributions=num_distributions, hidden_size=hidden_size)
opt = torch.optim.Adam(model1.parameters(), lr=1e-3)

for epoch in range(EPOCHS):
    losses = []
    for batch_x, batch_y in tqdm(train_dataloader1, desc=f'Epoch #{epoch + 1}'):
        pi, mu, sigma = model1(batch_x)
        opt.zero_grad()
        loss = calculate_loss(torch.unsqueeze(batch_y, 1), pi, mu, sigma, model1, lambda_pi)
        losses.append(loss.item())
        loss.backward()
        opt.step()
    print('Mean loss:', np.mean(losses))
    print('RMSE:', test_rmse(model1, test_dataset1, test_dataloader1).item())
    print()

Epoch #1: 100%|██████████████████████████████| 438/438 [00:00<00:00, 715.33it/s]


Mean loss: 2.296644694865022


Test: 100%|█████████████████████████████████| 117/117 [00:00<00:00, 3427.77it/s]


RMSE: 0.5971273183822632



Epoch #2: 100%|██████████████████████████████| 438/438 [00:00<00:00, 749.37it/s]


...

RMSE: 0.5861722230911255



Epoch #10: 100%|█████████████████████████████| 438/438 [00:00<00:00, 766.41it/s]


Mean loss: 0.26257005769341596


Test: 100%|█████████████████████████████████| 117/117 [00:00<00:00, 3214.22it/s]

RMSE: 0.5845715403556824



In [16]:
num_distributions = 3
hidden_size = 4
lambda_pi = 1
EPOCHS = 10

model5 = RMDN(num_distributions=num_distributions, hidden_size=hidden_size)
opt = torch.optim.Adam(model5.parameters(), lr=1e-3)

for epoch in range(EPOCHS):
    losses = []
    for batch_x, batch_y in tqdm(train_dataloader5, desc=f'Epoch #{epoch + 1}'):
        pi, mu, sigma = model5(batch_x)
        opt.zero_grad()
        loss = calculate_loss(torch.unsqueeze(batch_y, 1), pi, mu, sigma, model5, lambda_pi)
        losses.append(loss.item())
        loss.backward()
        opt.step()
    print('Mean loss:', np.mean(losses))
    print('RMSE:', test_rmse(model5, test_dataset5, test_dataloader5).item())
    print()

Epoch #1: 100%|██████████████████████████████| 438/438 [00:00<00:00, 552.56it/s]


Mean loss: 3.6001541000114727


Test: 100%|█████████████████████████████████| 117/117 [00:00<00:00, 2626.27it/s]


RMSE: 0.5778466463088989



Epoch #2: 100%|██████████████████████████████| 438/438 [00:00<00:00, 538.49it/s]


...

RMSE: 0.5760579109191895



Epoch #10: 100%|█████████████████████████████| 438/438 [00:00<00:00, 559.90it/s]


Mean loss: 0.23585541160951567


Test: 100%|█████████████████████████████████| 117/117 [00:00<00:00, 2699.16it/s]

RMSE: 0.5749793648719788



In [17]:
num_distributions = 3
hidden_size = 4
lambda_pi = 1
EPOCHS = 10

model10 = RMDN(num_distributions=num_distributions, hidden_size=hidden_size)
opt = torch.optim.Adam(model10.parameters(), lr=1e-3)

for epoch in range(EPOCHS):
    losses = []
    for batch_x, batch_y in tqdm(train_dataloader10, desc=f'Epoch #{epoch + 1}'):
        pi, mu, sigma = model10(batch_x)
        opt.zero_grad()
        loss = calculate_loss(torch.unsqueeze(batch_y, 1), pi, mu, sigma, model10, lambda_pi)
        losses.append(loss.item())
        loss.backward()
        opt.step()
    print('Mean loss:', np.mean(losses))
    print('RMSE:', test_rmse(model10, test_dataset10, test_dataloader10).item())
    print()

Epoch #1: 100%|██████████████████████████████| 437/437 [00:01<00:00, 408.09it/s]


Mean loss: 3.6531226758416784


Test: 100%|█████████████████████████████████| 116/116 [00:00<00:00, 2160.62it/s]


RMSE: 0.584004819393158



Epoch #2: 100%|██████████████████████████████| 437/437 [00:01<00:00, 414.93it/s]


...

RMSE: 0.576400876045227



Epoch #10: 100%|█████████████████████████████| 437/437 [00:01<00:00, 362.39it/s]


Mean loss: 0.2468990767768902


Test: 100%|█████████████████████████████████| 116/116 [00:00<00:00, 2119.92it/s]

RMSE: 0.5767474174499512



Применение для определение цены

In [22]:
THR = 1.5

mse = 0
num = 0
with torch.inference_mode():
    for batch_x, batch_y in tqdm(test_dataloader1):
        pi, mu, sigma = model1(batch_x)
        value = torch.sum(pi * mu, dim=1)  # expected values
        recomended = torch.exp(log_prob_y(pi, sigma, mu, torch.unsqueeze(value, 1))) > THR
        err = (batch_y - value)[recomended]
        mse += torch.sum(err**2)
        num += recomended.sum()
torch.sqrt(mse / num).item(), num.item() / len(test_dataset1)

100%|███████████████████████████████████████| 117/117 [00:00<00:00, 1883.28it/s]


(0.5268271565437317, 0.054691689008042894)

In [19]:
THR = 3

mse = 0
num = 0
with torch.inference_mode():
    for batch_x, batch_y in tqdm(test_dataloader5):
        pi, mu, sigma = model5(batch_x)
        value = torch.sum(pi * mu, dim=1)  # expected values
        recomended = torch.exp(log_prob_y(pi, sigma, mu, torch.unsqueeze(value, 1))) > THR
        err = (batch_y - value)[recomended]
        mse += torch.sum(err**2)
        num += recomended.sum()
torch.sqrt(mse / num).item(), num.item() / len(test_dataset5)

100%|███████████████████████████████████████| 117/117 [00:00<00:00, 1632.48it/s]


(0.5685878396034241, 0.25040300913487373)

In [20]:
THR = 3

mse = 0
num = 0
with torch.inference_mode():
    for batch_x, batch_y in tqdm(test_dataloader10):
        pi, mu, sigma = model10(batch_x)
        value = torch.sum(pi * mu, dim=1)  # expected values
        recomended = torch.exp(log_prob_y(pi, sigma, mu, torch.unsqueeze(value, 1))) > THR
        err = (batch_y - value)[recomended]
        mse += torch.sum(err**2)
        num += recomended.sum()
torch.sqrt(mse / num).item(), num.item() / len(test_dataset10)

100%|███████████████████████████████████████| 116/116 [00:00<00:00, 1395.44it/s]


(0.5197395086288452, 0.11961206896551724)